# Examples of how to use models

In [1]:
import os
import json

## 1. Huggingface AutoClass models for generating summaries

In [2]:
from summary_generation_with_autoclass import LLMGenerator
from huggingface_hub import login

/homes/sl318/.conda/envs/evaluation_mvp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# log into HuggingFace
# Replace hf_token with your own login token
with open("hf_token.txt","r") as f:
    hf_token = f.read().strip()
login(token=hf_token)

### Load example from CLPSYCH dataset

In [4]:
CLPSYCH_EXAMPLE = "example_data/clpsych_example.json"

with open(CLPSYCH_EXAMPLE, "r") as f:
    example = json.load(f)

In [5]:
# extract posts from example
posts = [post["post"] for post in example["posts"]]

### Initialize the LLM generator

In [ ]:
'''
initialise the LLM generator
you can either use a model from HuggingFace e.g. "meta-llama/Meta-Llama-3.1-8B-Instruct" 
or you can type "tulu" to use the temporal reasoning model that Jiayu developed
'''
summariser = LLMGenerator(
    model_name="tulu",
    cache_dir="/import/nlp-datasets/LLMs/", #Replace cache_dir with your own path
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.80s/it]
Device set to use cuda:0


Using device: cuda


### Define prompts

In [8]:
'''
for clspych we go through a process where we first summarise each post and then
we use the summarised posts to create the timeline summary
'''

prompts = [
            '''You are a helpful assistant. Your task is to analyze a post on self-states.

            # Definitions
            Self-states constitute identifiable units characterized by specific combinations of Affect, Behavior, Cognition, and Desire/Need that tend to be coactivated in a meaningful manner for limited periods of time. 
            An adaptive self-state pertains to aspects of Affect, Behaviour, and Cognition towards the self or others, which is conducive to the fulfillment of basic desires/needs, such as relatedness, autonomy and competence. 
            A maladaptive self-state pertains to aspects of Affect, Behaviour, and Cognition towards the self or others, that hinder the fulfillment of basic desires/needs.


            # Task Description
            Your response will have two parts.

            First, determine whether the post contains enough relevant information about the mental state of the individual. 
            Output YES if it contains relevant information, and NO otherwise.

            Then, write a summary in prose if there is relevant information, or return INSUFFICIENT INFORMATION.


            ## Example Output Format
            If there is enough information about the mental state:
            ```
            [START OUTPUT]
            YES
            <The summary goes here.>
            [END OUTPUT]
            ```

            Otherwise:
            ```
            [START OUTPUT]
            NO
            INSUFFICIENT INFORMATION
            [END OUTPUT]
            ```

            ## Summary Requirements
            If generating a summary, it should cover the following:
            - Identify whether there are adaptive and maladaptive self-states present. 
            - If they are both present, begin by determining which self-state is more dominant and describe it first.
            - If the self-state is maladaptive, explain how negative emotions, behaviors, or thoughts hinder psychological needs.
            - If the self-state is adaptive, explain how positive aspects support psychological needs.
            - If both adaptive and maladaptive states are present, describe each in turn. 
            - For each self-state, highlight the central organizing aspect—Affect, Behavior, Cognition, or Desire/Need—that drives the state. 
            - Describe how this central aspect influences the other aspects, focusing on the potential causal relationships between them. 

            The summary must be in a clear, concise paragraph that is strictly based on the input post content. 
            ''',
            '''You are a helpful assistant. Your task is to combine a sequence of self-state summaries.

                # Definitions
                Self-states constitute identifiable units characterized by specific combinations of Affect, Behavior, Cognition, and Desire/Need that tend to be coactivated in a meaningful manner for limited periods of time. 
                An adaptive self-state pertains to aspects of Affect, Behaviour, and Cognition towards the self or others, which is conducive to the fulfillment of basic desires/needs, such as relatedness, autonomy and competence. 
                A maladaptive self-state pertains to aspects of Affect, Behaviour, and Cognition towards the self or others, that hinder the fulfillment of basic desires/needs.

                # Instruction
                You will be given summaries describing self-states, ordered from oldest to most recent. 
                From these summaries, synthesize a meta-summary focusing on the interplay between adaptive and maladaptive self-states along the timeline. 
                Emphasize temporal dynamics focusing on concepts such as flexibility, rigidity, improvement, and deterioration. 
                If applicable, describe the extent to which the dominance of the self-states changes over time and how changes in aspects (Affect, Behavior, Cognition, and Desire) contribute to these transitions.

                The summary must be in a clear, concise paragraph that is strictly based on the input post content.
            '''
]

### Generate summary

In [11]:
# the LLMGenerator can handle the two step approach
# make sure to past prompts and max_tokens as lists of the same length
# the length of those lists represents the number of summarisation stages

summary = summariser.run_summary(
    prompt=prompts,
    text=posts,
    max_tokens=[700,500],
    temperature=0.0
)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.

In [12]:
print(summary)

 The post discusses the individual's experience with self-states, which are identifiable units characterized by specific combinations of Affect, Behavior, Cognition, and Desire/Need. The individual identifies both adaptive and maladaptive self-states within themselves. The maladaptive self-state is described as being characterized by negative emotions, behaviors, and thoughts that hinder the fulfillment of basic psychological needs such as relatedness, autonomy, and competence. This self-state is driven by a central organizing aspect of Affect, which influences the other aspects in a way that leads to negative emotions and behaviors. On the other hand, the adaptive self-state is characterized by positive aspects that support the fulfillment of psychological needs. This self-state is also driven by a central organizing aspect of Affect, but in a way that leads to positive emotions and behaviors. The individual's experience with these self-states highlights the importance of understandin